In [10]:
library(dplyr)
library(MuSiC) %>% suppressMessages()
library(fastSave) %>% suppressMessages()
library(pbmcapply) %>% suppressMessages()
library(Seurat) %>% suppressMessages()
library(SingleCellExperiment) %>% suppressMessages()

In [11]:
# load("~/代表性方法测试/BayesPrism/tutorial/tutorial.gbm.rdata")
combine.dgedata_150=readRDS.lbzip2('~/ReCIDE/benchmark测试_最终/deone_PAN/ref_data/ref_all.rdsFS',n.cores = 200)
patient_names=unique(combine.dgedata_150@meta.data['donor_id'])


In [15]:

func_mu<-function(i){
  print(paste('第',i,'个'))
  bulk_seurat<-subset(combine.dgedata_150,donor_id == patient_names[i,1])
    
  sc_seurat<-subset(combine.dgedata_150,donor_id != patient_names[i,1])

  bulk.mtx<-as.matrix(apply(as.data.frame(bulk_seurat@assays[["RNA"]]@counts),1,sum))
  bulk.mtx<-cbind(bulk.mtx,bulk.mtx)
  
  
  #单细胞参考集是sce的形式
  sce <- SingleCellExperiment(as.matrix(sc_seurat@assays$RNA@counts),
                              colData=DataFrame(label=sc_seurat@meta.data),
                              rowData=DataFrame(length=row.names(sc_seurat@assays$RNA@counts)))
  names(assays(sce))<-'counts'
  
  #在music前要先确定bulk和sc是否只有一列，只有一列的话无法运行（包括sample和cluster数是否为1）
  music_results_in = music_prop(bulk.mtx = bulk.mtx, sc.sce = sce, clusters = 'label.cell_label',
                                samples = 'label.donor_id',verbose = F)
    return(music_results_in)
}
music_results=pbmclapply(1:nrow(patient_names),func_mu,mc.cores=6)
names(music_results)<-patient_names[,1]

In [16]:
saveRDS(music_results,file='~/ReCIDE/benchmark测试_最终/deone_PAN/music/music_tutorial/music_output_parallel.rds')

In [17]:
# music_results_parallel=music_results

In [18]:
###结果整合
for (i in 1:length(music_results)) {
  music_results[[i]]<-music_results[[i]][['Est.prop.weighted']][1,]
}
prd<-music_results

df_merge<-as.data.frame(prd[[1]])
colnames(df_merge)[1]<-names(prd)[1]

for(j in 2:length(prd)){
  
  prd[[j]]<-as.data.frame(prd[[j]])
  df_merge<-merge(df_merge, prd[[j]], by = "row.names", all = TRUE)
  row.names(df_merge)<-df_merge[,1]
  df_merge<-df_merge[,-1]
  colnames(df_merge)[j]<-names(prd)[j]
}



df_merge<-as.data.frame(df_merge)
df_merge<-df_merge[,sort(colnames(df_merge))]
df_merge<-df_merge[sort(row.names(df_merge)),]
prd_df=df_merge

In [19]:
saveRDS(prd_df,file='~/ReCIDE/benchmark测试_最终/deone_PAN/music/music_tutorial/prd_music_df.rds')